### Package imports

In [1]:
import torch
from torchtext.legacy.data import Field, TabularDataset, BucketIterator
from torchtext.legacy import datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

import torch.optim as optim
import time
import random
import numpy as np
import torch.optim as optim

BATCH_SIZE = 10

#### parameters

In [80]:
BATCH_SIZE = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Data Processing

In [81]:
# step 1

removeTokens = lambda values: values[1:-1]   # function to remove [CLS] and [SEP] from the data set

tokens = Field(sequential=True,use_vocab=True,batch_first = True,preprocessing=removeTokens,lower=True,pad_token="0")
edits = Field(sequential=True,use_vocab=True,batch_first = True,preprocessing=removeTokens,lower=True,pad_token="0")

fields = {'tokens':('tokens',tokens),'labels':('edits',edits)}

train_data, test_data = TabularDataset.splits(path='data',train='ptrain.jsonl',test='val.jsonl',
                                              format='json',fields=fields)

# train_data is dataset with edits and tokens pair. in edits and tokens list of string is available

In [82]:
print(train_data)              # Tabular Data set object

print(train_data[0].__dict__.keys())
print(train_data[0].__dict__.values())

dict_keys(['tokens', 'edits'])
dict_values([['plus', ',', 'the', 'novelty', 'of', 'the', 'iphone', 'won', "'t", 'wear', 'off', ',', 'as', 'it', 'may', 'with', 'a', 'camcorder', ';', 'and', 'over', 'these', 'video', 'apps', 'have', 'fun', 'effects', 'that', 'a', 'camcorder', 'can', "'t", 'match', '.'], ['$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$replace_might', '$keep', '$keep', '$keep', '$replace_,', '$keep', '$delete', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep']])


In [83]:
train_data.examples[1].edits    # to print tokens of any row

['$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$keep',
 '$transform_verb_vbz_vb',
 '$keep',
 '$append_.']

In [84]:
# step 2  Build a vocab

tokens.build_vocab(train_data,min_freq=1,vectors='glove.6B.100d')
edits.build_vocab(train_data,min_freq=1,vectors='glove.6B.100d')

In [85]:
print(tokens)   # field object
print(tokens.vocab)   #vocab object
print(train_data[0].__dict__.keys())
print(train_data[0].__dict__.values())
print(len(train_data[0].__dict__.values()))

# Note: even after the building vocab the train data will not change

dict_keys(['tokens', 'edits'])
dict_values([['plus', ',', 'the', 'novelty', 'of', 'the', 'iphone', 'won', "'t", 'wear', 'off', ',', 'as', 'it', 'may', 'with', 'a', 'camcorder', ';', 'and', 'over', 'these', 'video', 'apps', 'have', 'fun', 'effects', 'that', 'a', 'camcorder', 'can', "'t", 'match', '.'], ['$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$replace_might', '$keep', '$keep', '$keep', '$replace_,', '$keep', '$delete', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep']])
2


In [86]:
# Step 3 (Create a iterator to loop over the data. Also separate batchs with 
#         similar length and pad the extra space)

train_data_iterator, test_data_iterator = BucketIterator.splits((train_data,test_data),
                                                                batch_size=BATCH_SIZE)

In [87]:
batch_1 = iter(train_data_iterator)
batch_1 = next(batch_1)
print(batch_1.tokens.shape)
print(batch_1.edits[0])

torch.Size([1, 16])
tensor([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 22,  2,  7])


In [10]:
#string to index
print(f"tokens.vocab.stoi['$$$'] = {tokens.vocab.stoi['$$$']}")
print(f"tokens.vocab.itos[0] = {tokens.vocab.itos[0]}")
print(f"edits.vocab.stoi['$keep'] = {edits.vocab.stoi['$keep']}")
print(f"edits.vocab.itos[2] = {edits.vocab.itos[2]}")
print()

#length of vocabular create from the data set
print(f"len(tokens.vocab) = {len(tokens.vocab)}")
print(f"len(edits.vocab) = {len(edits.vocab)}")
print()

#shape of vocabular create from the data set
print(f"tokens.vocab.vectors.shape = {tokens.vocab.vectors.shape}")
print(f"edits.vocab.vectors.shape = {edits.vocab.vectors.shape}")
print()

# no. of unique words in tokens and edits
print(f"len(tokens.vocab.freqs.keys()) = {len(tokens.vocab.freqs.keys())}")
print(f"len(edits.vocab.freqs.keys()) = {len(edits.vocab.freqs.keys())} \n")
print(f"edits.vocab.freqs = {edits.vocab.freqs}")

tokens.vocab.stoi['$$$'] = 0
tokens.vocab.itos[0] = <unk>
edits.vocab.stoi['$keep'] = 2
edits.vocab.itos[2] = $keep

len(tokens.vocab) = 186
len(edits.vocab) = 23

tokens.vocab.vectors.shape = torch.Size([186, 100])
edits.vocab.vectors.shape = torch.Size([23, 100])

len(tokens.vocab.freqs.keys()) = 184
len(edits.vocab.freqs.keys()) = 21 

edits.vocab.freqs = Counter({'$keep': 270, '$delete': 11, '$replace_.': 5, '$transform_agreement_singular': 2, '$append_every': 2, '$replace_might': 1, '$replace_,': 1, '$transform_verb_vbz_vb': 1, '$append_.': 1, '$transform_case_lower': 1, '$append_had': 1, '$replace_the': 1, '$append_said': 1, '$replace_no': 1, '$transform_verb_vbn_vbg': 1, '$replace_by': 1, '$append_yesterday': 1, '$replace_are': 1, '$replace_so': 1, '$append_for': 1, '$replace_what': 1})


build vocab just takes unique tokens from the dataset and given a position and stores
as a dictionary. when it is applied to the dataset the result comming from the 
bucket iteartor is just a postion no. from the build vocab and the rest is padded

### Model

In [69]:
class Seq_to_edits(nn.Module):
    def __init__(self,token_vocab_size : int        # num of unique token in the entire dataset 
               ,edit_vocab_size : list             # [batch_size,max sentence size in the batch]
               ,output_dim : list            # [batch_size,max sentence size in the batch]
               ,embed_dim : int              # embedding dimension
               ,num_of_filters : int         # to num of filters
               ,filter_sizes : list          # list of sizes of each filter
               ,dropout : int                # dropout percent ()
               ,pad_index : int              # padding index from the vocab
               ,max_lenght : int = 100):     # Maximum words in a list of tokens or sentence
               
        
        
        super().__init__()
        self.device = device
        
        # Embedding layer
        self.token_embedding_layer = nn.Embedding(vocab_size,embed_dim)
        self.position_embedding_layer = nn.Embedding(max_lenght,embed_dim)
        
        # Multiple conv 1D layers 
        self.conv_layers = nn.ModuleList([nn.Conv1d(in_channels = embed_dim,
                                                    out_channels= embed_dim,
                                                    kernel_size = filter_size,
                                                    stride = filter_size) 
                                          for filter_size in filter_sizes])
        
        #self.final_layer = nn.Conv1d(in_channels = embed_dim*len(filter_sizes),out_channels=output_dim,kernel_size = 1)
        
        # Dropout layers
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, tokens):
        
        # tokens size is [batch_size, max sentence size in the batch/token size]
        
        batch_size , num_of_tokens = tokens.shape
        
        position = torch.arange(0,num_of_tokens).unsqueeze(0)         # [1,num_of_tokens]
        position = position.repeat(batch_size,1).to(self.device)    # [batch_size, num_of_tokens]
        
        # apply embedding layer
        token_embeddings    = self.token_embedding_layer(tokens)
        position_embeddings = self.position_embedding_layer(position)
        # token_embeddings = position_embeddings = [batch_size, num_of_tokens,embedding dimension]
        
        combination = self.dropout(token_embeddings+position_embeddings)
        # combination = [batch_size, num_of_tokens,embedding dimension]
        
        conv_input = combination.reshape(batch_size,-1).unsqueeze(1)
        # conv_input = [batch_size, embedding dimension, num_of_tokens]
        
        
        conved = [F.relu(conv_layer(conv_input)).squeeze(1) for conv_layer in self.conv_layers]
        # conved = [batch_size, embedding dimension, num_of_tokens]
        
        output = torch.cat(conved,dim=1)
        
        return output 
        

### HyperParameter

In [38]:
learning_rate = 0.0001
num_epochs = 10
INPUT_DIM = len(tokens.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 3
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = len(edits.vocab)
DROPOUT = 0.5
PAD_IDX = tokens.vocab.stoi[tokens.pad_token]
UNK_IDX = tokens.vocab.stoi[tokens.unk_token]

### Initialize model

In [39]:
# model is created

model = Seq_to_edits(INPUT_DIM,OUTPUT_DIM,EMBEDDING_DIM,N_FILTERS,FILTER_SIZES,DROPOUT,PAD_IDX)

In [40]:
# trainable parameters are printed

count_parameters= lambda model:sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 155,823 trainable parameters


In [41]:
# load the pre-tained embeddings 

pretrained_embeddings = tokens.vocab.vectors

model.token_embedding_layer.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.6149,  0.9273,  0.5583,  ..., -0.0922, -0.0798, -1.2843],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4221,  0.6307, -0.1291,  ...,  0.1045,  0.1854, -0.4767],
        [ 0.2783,  0.4995,  0.2494,  ..., -0.3274, -0.3004,  0.0930],
        [ 0.7090,  0.6810, -1.2128,  ...,  0.6917,  1.4080, -1.0227]])

In [42]:
# initialize zero weights for unknown and padding tokens.


model.token_embedding_layer.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.token_embedding_layer.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [46]:
# Initialize optimizer and loss function

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_data_iterator)
model = model.to(device)
criterion = criterion.to(device)
n_total_steps = len(train_data_iterator)

### model train and test

In [47]:
# Ini

### Model(rough)

In [50]:

for epoch in range(num_epochs):
    for i, batch in enumerate(train_data_iterator):
        optimizer.zero_grad()


        # converting to cpu or gpu variable
        tokens_list = batch.tokens.to(device)
        edits_list = batch.edits.to(device)

        # Forward pass
        outputs = model(tokens_list)
        loss = criterion(outputs, edits_list)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')



print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)


RuntimeError: The size of tensor a (100) must match the size of tensor b (5) at non-singleton dimension 1

In [49]:
model.eval() 
epoch_loss = 0 
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for batch in test_data_iterator:
        tokens_list = batch.tokens.to(device)
        edits_list = batch.edits.to(device)
        
        outputs = model(tokens_list)
        
        predictions = model(batch.text).squeeze(1)
            
        loss = criterion(predictions, batch.edits_list)
        
        epoch_loss += loss.item()
print(f"avg loss {epoch_loss/n_total_steps}")

TypeError: '<' not supported between instances of 'Example' and 'Example'

### Rough work

In [ ]:
l=['plus', ',', 'the', 'novelty', 'of', 'the', 'iphone', 'won', "'t", 'wear', 'off', ',', 'as', 'it', 'may', 'with', 'a', 'camcorder', ';', 'and', 'over', 'these', 'video', 'apps', 'have', 'fun', 'effects', 'that', 'a', 'camcorder', 'can', "'t", 'match', '.']
l1=['$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$replace_might', '$keep', '$keep', '$keep', '$replace_,', '$keep', '$delete', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep', '$keep']
print(len(l))
print(len(l1))

In [ ]:
n_embeddings, dim = 10,4 #10 - # of vocac size 4 - # of emdebbing dimension

In [ ]:
m = nn.Conv1d(1, 2, 3)
input = torch.randn(2, 1, 5) # (batch size, no. of channel, # of words)
# x = emb_1(input)
output = m(input)

In [ ]:
input.shape

In [ ]:
output.shape

In [ ]:
from torch.nn import Embedding as emb

emb_1 = emb(188, 50)
print(f"embedding {emb_1}")
print(f"embedding weight's shape {emb_1.weight.shape}") #requires_grad=True therefore the matrix is learnable

print(f"values of weight {emb_1.weight}")

for batch in train_data_iterator:
    x = emb_1(batch.tokens)

In [ ]:
x.shape

In [ ]:
x = torch.tensor([[1,2,4],[1, 2, 3]])
print(x.shape)
x=x.repeat(4, 2)
print(x.shape)


In [ ]:
import torch
N = 40
C_in = 40
L_in = 100
inputs = torch.rand([N, C_in, L_in])
padding = 3
kernel_size = 3
stride = 2
C_out = 10
x = torch.nn.Conv1d(C_in, C_out, kernel_size, stride=stride, padding=padding)
y = x(inputs)
print(y)
print(y.shape)

In [73]:
x_input = torch.randn([2,4,5])
# print(x_input)
x_input = x_input.reshape(2,-1).unsqueeze(1)
print(x_input.shape)
print(x_input)

torch.Size([2, 1, 20])
tensor([[[-1.3545,  0.6273, -1.1048,  1.5357,  1.0720,  1.7085,  1.8519,
          -0.6226, -0.2996, -1.9516, -0.0344,  0.2025, -0.4796,  1.0473,
           0.5128,  0.8766,  0.2924,  1.6486, -0.0163,  1.0972]],

        [[ 0.4953,  0.9346,  0.1665,  0.2923,  1.4549, -0.3342,  1.7224,
           0.6442, -0.6272, -0.2344, -0.0523, -1.8436, -0.5091, -0.3261,
           0.2669,  1.6999, -0.7664, -0.2777, -1.1153,  1.2729]]])


In [106]:
a = nn.Conv1d(in_channels =1 ,out_channels= 1,kernel_size = 2,stride=2)(x_input)
b = nn.Conv1d(in_channels =1 ,out_channels= 1,kernel_size = 1,stride=1)(x_input)
c = nn.Conv1d(in_channels =1 ,out_channels= 1,kernel_size = 3,stride=3)(x_input)
d = nn.Conv1d(in_channels =1 ,out_channels= 1,kernel_size = 4,stride=4)(x_input)
d = nn.Conv1d(in_channels =1 ,out_channels= 1,kernel_size = 5,stride=5)(x_input)

In [107]:
print(f"a = {a.shape}")
print(f"b = {b.shape}")
print(f"c = {c.shape}")
print(f"d = {d.shape}")

a = torch.Size([2, 1, 10])
b = torch.Size([2, 1, 20])
c = torch.Size([2, 1, 6])
d = torch.Size([2, 1, 4])


In [108]:
pooled = [a.squeeze(1),b.squeeze(1),c.squeeze(1),d.squeeze(1)]
e = torch.cat(pooled,dim=1)

In [109]:
print(f"e = {e.shape}")

e = torch.Size([2, 40])


In [ ]:
print(f"a1 = {a1.shape}")
print(f"b1 = {b1.shape}")
print(f"c1 = {c1.shape}")
print(f"d1 = {d1.shape}")

In [ ]:
a1 = nn.Conv1d(in_channels =4 ,out_channels= 4,kernel_size = 2,padding="same")(x_input)
b1 = nn.Conv1d(in_channels =4 ,out_channels= 4,kernel_size = 1,padding="same")(x_input)
c1 = nn.Conv1d(in_channels =4 ,out_channels= 4,kernel_size = 3,padding="same")(x_input)
d1 = nn.Conv1d(in_channels =4 ,out_channels= 4,kernel_size = 4,padding="same")(x_input)
e1 = torch.cat([a1,b1,c1,d1],dim =1)
f1 = nn.Conv1d(in_channels = e1.shape[1],out_channels= 25,kernel_size = 1)(e1)

In [ ]:
pooled1 = [a1,b1,c1,d1]


In [ ]:
print(f"e1 = {e1.shape}")

In [ ]:
print(f1.shape)